### Step 1. 필요한 라이브러리 임포트

In [1]:
import pandas as pd
import re

### Step 2. [데이터](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv) 확인

- order_id 주문번호
- quantity 주문 개수
- item_name 주문한 상품명
- choice_description 선택사항
- item_price 가격

### Step 3. 데이터 불러오기


**read_csv를 통해 데이터를 올바른 형태로 불러온 뒤 chipo에 저장해주세요**  
data: chipotle.tsv

In [ ]:
chipo=pd.read_csv('../data/chipotle.tsv', sep='\t')
chipo.head()

In [ ]:
chipo.info()

### Step 4. 누락값 확인하기
chipo에서 누락값이 존재하는지 확인해주세요

In [ ]:
chipo.info()

In [ ]:
chipo.choice_description.isnull().value_counts()

In [ ]:
# 누락값을 리스크로 넣는 방법--문자열로 넣어주면 됨
chipo.choice_description.fillna('[]')

### Step 5. item_price에서 $를 없애고 float 형식으로 새롭게 저장해주세요.

In [ ]:
# $ 없애기 방법1)
chipo['item_price']=chipo['item_price'].str.replace(pat=r'[^\w]', repl=r'',regex=True)
chipo['item_price']

In [ ]:
# 없애기 방법2
def dollar_del(x):
    return float(x.replace('$',''))
    
chipo['item_price']=chipo['item_price'].apply(dollar_del)

In [ ]:
# 타입 변경
chipo_f=chipo.item_price.astype('float')
chipo_f

### Step 6. 아이템별 주문 수를 출력해주세요

In [ ]:
chipo.groupby('item_name')['order_id'].size()  
# size: 누락값 포함하므로 가장 좋음// count는 누라값 제외하므로 정확하지 않을 수 있음

### Step 7. 아이템별 주문 총량을 출력해주세요.

In [ ]:
chipo.groupby('item_name')['quantity'].sum().sort_values(ascending=False)

In [ ]:
chipo.head()

### Step 8. 각 아이템의 가격을 구해서 unit_price 컬럼에 추가해주세요.
주문당 quantity가 1일때의 가격 (item_price / quantity)

In [ ]:
chipo.info()

In [122]:
chipo['unit_price']= chipo['item_price'] / chipo['quantity']

In [128]:
chipo

,order_id,quantity,item_name,choice_description,item_price,unit_price
0,1,1,Chips and Fresh Tomato Salsa,NaN,2.39,2.39
1,1,1,Izze,[Clementine],3.39,3.39
2,1,1,Nantucket Nectar,[Apple],3.39,3.39
3,1,1,Chips and Tomatillo-Green Chili Salsa,NaN,2.39,2.39
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49
...,...,...,...,...,...,...
4617,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Black Beans, Sour ...",11.75,11.75
4618,1833,1,Steak Burrito,"[Fresh Tomato Salsa, [Rice, Sour Cream, Cheese...",11.75,11.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75,8.75


### Step 9. 아이템별 개당 가격의 평균을 구하고 가장 비싼순으로 5개를 출력해주세요

In [133]:
# 아이템 별  --> groupby 해줘야
chipo.groupby('item_name')['unit_price'].mean().sort_values(ascending=False).head()  

item_name
Carnitas Salad Bowl    11.056667
Steak Salad Bowl       11.027931
Barbacoa Salad Bowl    10.640000
Carnitas Bowl          10.315588
Steak Bowl             10.232559
Name: unit_price, dtype: float64

### Step 10. 한 주문(같은 order_id)에 20달러 이상 지불한 손님을 출력해주세요.

In [154]:
# 그룹을 만든다움에는 인덱스가 없어짐/바꺄서 안들어감 ==> 두 문장으로 나누어서 작업하기
chipo_20=chipo.groupby('order_id')['item_price'].sum()
chipo_order=chipo_20[chipo_20 >= 20].index     # 손님을 출력해야 
chipo_order

Index([   4,   18,   20,   21,   26,   28,   30,   34,   40,   43,
       ...
       1811, 1812, 1822, 1824, 1825, 1827, 1829, 1830, 1833, 1834],
      dtype='int64', name='order_id', length=589)

### Step 11. 위 10번에서 주문량이 2개인 아이템중 가장 많이 팔린 아이템은 무엇인가요?
Chips and Guacamole

In [160]:
# order_id로 인덱스를 찾아서 고객정보 찾아야/위 결과의 인덱스를 order_id컬럼에서 찾아라
chipo20_df=chipo[chipo['order_id'].isin(chipo_order)]

In [163]:
chipo_df2=chipo20_df[chipo20_df['quantity'] == 2]
chipo_df2.groupby('item_name').size().sort_values(ascending=False)

item_name
Chips and Guacamole                      20
Chicken Bowl                             17
Canned Soft Drink                        15
Chicken Burrito                          13
Chips                                    12
Steak Burrito                            11
Chicken Salad Bowl                       10
Canned Soda                               7
Bottled Water                             6
Steak Bowl                                6
Chicken Crispy Tacos                      3
Chicken Soft Tacos                        3
Chips and Fresh Tomato Salsa              2
Chips and Tomatillo Green Chili Salsa     2
Chips and Tomatillo Red Chili Salsa       2
Steak Salad Bowl                          2
Nantucket Nectar                          2
Steak Soft Tacos                          1
Veggie Bowl                               1
Steak Crispy Tacos                        1
6 Pack Soft Drink                         1
Chips and Tomatillo-Red Chili Salsa       1
Chips and Tomatillo-Gr

### Step 12. Veggie Salad Bowl은 몇 건 주문되었나요?

In [1]:
chipo.order_id[chipo['item_name'] == 'Veggie Salad Bowl'].size()

NameError: name 'chipo' is not defined

### Step 13. 아이템명에 Chicken이 들어간 음식은 몇 건 주문되었나요?
**Hint: 정규표현식 사용**   
1560

In [ ]:
# 정규표현식 모듈 re 사용
print(re.match('Chicken','Chicken Bow' ))  # 맨 앞에 있어야
print(re.search('Chicken','Chicken Bow' ))  # 어디에 있던지 상관 없음

In [166]:
result=chipo[chipo['item_name'].str.contains('Chicken')]
result

,order_id,quantity,item_name,choice_description,item_price,unit_price
4,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans...",16.98,8.49
5,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sou...",10.98,10.98
11,6,1,Chicken Crispy Tacos,"[Roasted Chili Corn Salsa, [Fajita Vegetables,...",8.75,8.75
12,6,1,Chicken Soft Tacos,"[Roasted Chili Corn Salsa, [Rice, Black Beans,...",8.75,8.75
13,7,1,Chicken Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Rice,...",11.25,11.25
...,...,...,...,...,...,...
4604,1828,1,Chicken Bowl,"[Fresh Tomato Salsa, [Rice, Black Beans, Chees...",8.75,8.75
4615,1832,1,Chicken Soft Tacos,"[Fresh Tomato Salsa, [Rice, Cheese, Sour Cream]]",8.75,8.75
4619,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Pinto...",11.25,11.25
4620,1834,1,Chicken Salad Bowl,"[Fresh Tomato Salsa, [Fajita Vegetables, Lettu...",8.75,8.75


In [ ]:
# apply 사용할 것

def chicken_in(x):
    sum=0
    if 'Chicken' in x:
        sum=sum+1
    return sum
    
chipo[chipo['item_name'].apply(chicken_in)
    